In [1]:
!pip install "numpy<2.0.0"


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
%%capture
!uv pip install openpipe-art openpipe --prerelease allow --no-cache-dir

In [9]:
!pip install -q unsloth

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.6.0 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
!pip install -q dotenv 


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
!pip install --upgrade typing-extensions>=4.6.1



[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install pygame

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 44.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:

import random
import string
import xml.etree.ElementTree as ET
from typing import TypedDict, Literal, List, Optional
from dotenv import load_dotenv 
import torch
from unsloth import FastLanguageModel


load_dotenv()


WINNING_VALUE = 2048  
BOARD_SIZE = 4        


RESET = "\033[0m"

COLORS = {
    None: "38;5;240;48;5;236",  
    2:    "38;5;232;48;5;231",  
    4:    "38;5;232;48;5;229",  
    8:    "38;5;231;48;5;215",  
    16:   "38;5;231;48;5;209",  
    32:   "38;5;231;48;5;203",  
    64:   "38;5;231;48;5;196",  
    128:  "38;5;232;48;5;228",  
    256:  "38;5;232;48;5;227",  
    512:  "38;5;232;48;5;226",  
    1024: "38;5;231;48;5;220",  
    2048: "38;5;231;48;5;214",  
   
    'default': "38;5;231;48;5;160", 
}


BOX_TOP_LEFT = "┌"
BOX_TOP_RIGHT = "┐"
BOX_BOTTOM_LEFT = "└"
BOX_BOTTOM_RIGHT = "┘"
BOX_HORIZONTAL = "─"
BOX_VERTICAL = "│"
BOX_CROSS = "┼"
BOX_T_DOWN = "┬"
BOX_T_UP = "┴"
BOX_T_LEFT = "┤"
BOX_T_RIGHT = "├"

TILE_WIDTH = 6 






class TwentyFortyEightGame(TypedDict):
    id: str
    board: List[List[Optional[int]]]
    board_size: int 


def populate_random_cell(game: TwentyFortyEightGame) -> bool:
    """Populates a random empty cell. Returns True if successful, False if no empty cells."""
    all_clear_coordinates = [
        (i, j)
        for i in range(game["board_size"])
        for j in range(game["board_size"])
        if game["board"][i][j] is None
    ]
    if not all_clear_coordinates:
        return False 

    random_clear_coordinates = random.choice(all_clear_coordinates)
    
    game["board"][random_clear_coordinates[0]][random_clear_coordinates[1]] = (
        2 if random.random() < 0.9 else 4
    )
    return True


def generate_game(board_size: int = BOARD_SIZE) -> TwentyFortyEightGame:
   
    id = "".join(random.choices(string.ascii_letters + string.digits, k=6))
    game: TwentyFortyEightGame = {
        "id": id,
        "board": [[None for _ in range(board_size)] for _ in range(board_size)],
        "board_size": board_size,
    }

   
    populate_random_cell(game)
    populate_random_cell(game)

    return game


def condense_sequence(sequence: List[Optional[int]], size: int) -> List[Optional[int]]:
    """Condenses a single row/column according to 2048 rules."""
    condensed_sequence = []
    score_increase = 0 

    gapless_sequence = [cell for cell in sequence if cell is not None]

    i = 0
    while i < len(gapless_sequence):
        if (
            i + 1 < len(gapless_sequence)
            and gapless_sequence[i] == gapless_sequence[i + 1]
        ):
            merged_value = gapless_sequence[i] * 2
            condensed_sequence.append(merged_value)
            score_increase += merged_value 
            i += 2
        else:
            condensed_sequence.append(gapless_sequence[i])
            i += 1


    return condensed_sequence + [None] * (size - len(condensed_sequence)) 



def condense_board(
    game: TwentyFortyEightGame, direction: Literal["left", "right", "up", "down"]
) -> bool:
    """
    Condenses the board in the given direction.
    Returns True if the board changed, False otherwise.
    """
    board_size = game["board_size"]
    original_board_repr = repr(game["board"]) 

    if direction == "left":
        for r in range(board_size):
            row = game["board"][r]
            condensed_row = condense_sequence(row, board_size)
            game["board"][r] = condensed_row

    elif direction == "right":
        for r in range(board_size):
            row = game["board"][r]
            reversed_row = row[::-1]
            condensed_reversed = condense_sequence(reversed_row, board_size)
            game["board"][r] = condensed_reversed[::-1] 

    elif direction == "up":
        for c in range(board_size):
            column = [game["board"][r][c] for r in range(board_size)]
            condensed_column = condense_sequence(column, board_size)
            for r in range(board_size):
                game["board"][r][c] = condensed_column[r]

    elif direction == "down":
        for c in range(board_size):
            column = [game["board"][r][c] for r in range(board_size)]
            reversed_column = column[::-1]
            condensed_reversed = condense_sequence(reversed_column, board_size)
            final_column = condensed_reversed[::-1] 
            for r in range(board_size):
                game["board"][r][c] = final_column[r]

    return repr(game["board"]) != original_board_repr


def render_board(game: TwentyFortyEightGame) -> str:
    board = game["board"]
    board_size = game["board_size"]
    board_str = "\n" 

    
    def get_color_style(value):
        style_code = COLORS.get(value)
        if style_code is None and value is not None: 
             style_code = COLORS.get('default', "0") 
        elif style_code is None: 
             style_code = COLORS.get(None, "0") 
        return f"\033[{style_code}m"

   
    board_str += BOX_TOP_LEFT + (BOX_HORIZONTAL * TILE_WIDTH + BOX_T_DOWN) * (board_size - 1) + BOX_HORIZONTAL * TILE_WIDTH + BOX_TOP_RIGHT + "\n"

    
    for i, row in enumerate(board):
        row_render = BOX_VERTICAL
        for cell in row:
            value_str = str(cell) if cell is not None else ""
            
            centered_val = value_str.center(TILE_WIDTH)
            color_style = get_color_style(cell)
            
            row_render += f"{color_style}{centered_val}{RESET}{BOX_VERTICAL}"
        board_str += row_render + "\n"

      
        if i < board_size - 1:
            board_str += BOX_T_RIGHT + (BOX_HORIZONTAL * TILE_WIDTH + BOX_CROSS) * (board_size - 1) + BOX_HORIZONTAL * TILE_WIDTH + BOX_T_LEFT + "\n"
        else:
          
            board_str += BOX_BOTTOM_LEFT + (BOX_HORIZONTAL * TILE_WIDTH + BOX_T_UP) * (board_size - 1) + BOX_HORIZONTAL * TILE_WIDTH + BOX_BOTTOM_RIGHT + "\n"

    return board_str



def apply_agent_move(game: TwentyFortyEightGame, move_xml: str) -> bool:
    """
    Applies the agent's move.
    Returns True if the move was valid and changed the board, False otherwise.
    Raises ValueError on invalid XML or direction.
    """
    direction = None
   
    try:
        
        move_xml = move_xml.strip()
        if not move_xml.startswith("<move>") or not move_xml.endswith("</move>"):
            
             start = move_xml.find("<move>")
             end = move_xml.find("</move>")
             if start != -1 and end != -1:
                 move_xml = move_xml[start:end+len("</move>")]
             else:
                  raise ValueError("Invalid XML structure")

        root = ET.fromstring(move_xml)
        direction_text = root.text
        if direction_text:
             direction = direction_text.strip().lower()
        else:
             raise ValueError("Move tag is empty")

    except ET.ParseError:
        raise ValueError(f"Invalid XML: {move_xml}")
    except Exception as e:
        print(f"Error parsing move XML: {e}") 
        raise ValueError("Error processing move XML")


    if direction not in ["left", "right", "up", "down"]:
        raise ValueError(f"Invalid direction: '{direction}'")


    board_changed = condense_board(game, direction)

 
    if board_changed:
        populate_random_cell(game)
        return True
    else:
        
        return False 



def max_cell_value(game: TwentyFortyEightGame) -> int:
    max_val = 0
    for row in game["board"]:
        for cell in row:
            if cell is not None and cell > max_val:
                max_val = cell
    return max_val


def check_moves_possible(game: TwentyFortyEightGame) -> bool:
    """Checks if any move (left, right, up, down) would change the board state."""
    board_size = game["board_size"]
    board = game["board"]

 
    for r in range(board_size):
        for c in range(board_size - 1):
            if board[r][c] == board[r][c+1] or board[r][c] is None or board[r][c+1] is None:
                return True 
    for c in range(board_size):
        for r in range(board_size - 1):
            if board[r][c] == board[r+1][c] or board[r][c] is None or board[r+1][c] is None:
                 return True 
    if any(board[r][board_size-1] is None for r in range(board_size)): return True
    if any(board[board_size-1][c] is None for c in range(board_size)): return True


    return False 



def check_game_finished(game: TwentyFortyEightGame) -> bool:
    if max_cell_value(game) >= WINNING_VALUE:
        return True # Won

    
    if any(cell is None for row in game["board"] for cell in row):
        return False

    
    if not check_moves_possible(game):
        return True 

    return False 
def total_board_value(game: TwentyFortyEightGame) -> int:
    return sum([cell for row in game["board"] for cell in row if cell is not None])




model_name = "Jnaranjo/2048-player-model"
try:
    peft_model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=16384, 
        dtype=torch.bfloat16, 
        load_in_4bit=True,    
    )
    FastLanguageModel.for_inference(peft_model)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have the necessary libraries and model access.")
    exit()



game = generate_game(board_size=BOARD_SIZE)
move_number = 0
max_invalid_moves = 5 
invalid_move_count = 0


messages = [
    {
        "role": "system",
        "content": f"You are an expert 2048 player. Your goal is to combine tiles to reach the {WINNING_VALUE} tile. "
                   "Analyze the board state and choose the optimal move. "
                   "Valid moves are 'left', 'right', 'up', 'down'. "
                   f"Return ONLY the chosen move as an XML object with a single 'move' element, like this: <move>left</move>. Do not add any other text."
    }
]

print("Starting game...")
print(render_board(game))

# Game loop
while not check_game_finished(game):
    if invalid_move_count >= max_invalid_moves:
        print(f"Agent made {max_invalid_moves} consecutive invalid or non-progressing moves. Ending game.")
        break

    rendered_board = render_board(game)

    user_prompt = f"Current board state:\n{rendered_board}\n\nChoose your next move (left, right, up, down)."
    messages.append({"role": "user", "content": user_prompt})


    try:
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True, 
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(peft_model.device) 
    except Exception as e:
        print(f"Error preparing input for tokenizer: {e}")
       
        print("Messages causing error:")
        for msg in messages:
            print(f"  Role: {msg['role']}, Content Length: {len(msg['content'])}")
       
        if len(messages) > 10: 
             messages = messages[:1] + messages[-8:]
             print("Truncated message history.")
            
             inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(peft_model.device)
        else:
            raise e 
    agent_move_xml = None
    try:
        with torch.no_grad():
            outputs = peft_model.generate(
                input_ids=inputs,
                max_new_tokens=20,  
                do_sample=False,    
                # temperature=0.1,  
                # top_p=0.9,
                pad_token_id=tokenizer.eos_token_id 
            )
          
            decoded_output = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
            agent_move_xml = decoded_output.strip()

    except Exception as e:
        print(f"\nError during model generation on move {move_number + 1}: {e}")
       
        if "out of memory" in str(e).lower():
            print("CUDA out of memory. Try reducing batch size (if applicable) or model size.")
        
        break 


    messages.append({"role": "assistant", "content": agent_move_xml})

   
    try:
        move_successful = apply_agent_move(game, agent_move_xml)
        if move_successful:
            move_number += 1
            invalid_move_count = 0 
            print(f"\n--- Move {move_number} ---")
            
            print(f"Agent suggested move: {agent_move_xml}")
            print(f"Board after move:")
            print(render_board(game))
        else:
       
            print(f"\nMove {move_number + 1}: Agent suggested '{agent_move_xml}', but it didn't change the board. Asking again.")
          
            messages.pop() 
            messages.pop() 
            invalid_move_count += 1
            

    except ValueError as e:
        print(f"\nInvalid move attempt on move {move_number + 1}: {e}")
        print(f"Agent raw output: '{agent_move_xml}'")
        
        messages.pop() 
        messages.pop() 
        invalid_move_count += 1
        
        messages.append({"role": "user", "content": "That was not a valid move format. Please return only <move>direction</move> where direction is left, right, up, or down."})
        print("Asking agent for a valid move...")
      


    if len(messages) > 11:
        messages = messages[:1] + messages[-10:]



print("\n====================")
print(f"Game finished in {move_number} valid moves.")
print("====================")

final_board_render = render_board(game)
max_val = max_cell_value(game)
board_val = total_board_value(game)

print(f"Final board:\n{final_board_render}")
print(f"Max tile value reached: {max_val}")
print(f"Total board value: {board_val}")

if max_val >= WINNING_VALUE:
    print("\nCongratulations! Agent reached the target value! 💪🏆")
elif invalid_move_count >= max_invalid_moves:
     print("\nGame ended due to repeated invalid moves. 🥶")

    print("\nGame over! No more possible moves. 😢")

Loading AI model...
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA RTX A5000. Num GPUs = 1. Max memory: 23.573 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model loaded successfully.
Starting game...

┌──────┬──────┬──────┬──────┐
│      │      │      │  2   │
├──────┼──────┼──────┼──────┤
│      │      │      │      │
├──────┼──────┼──────┼──────┤
│      │      │      │      │
├──────┼──────┼──────┼──────┤
│      │      │      │  2   │
└──────┴──────┴──────┴──────┘


--- Move 1 ---
Agent suggested move: <move>up</move>
Board after move:

┌──────┬──────┬──────┬──────┐
│      │      │      │  4   │
├──────┼──────┼──────┼──────┤
│      │      │      │      │
├──────